In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

import tree as t
import player as p

In [ ]:
def My_NN_train(X_train,y_train):

	regressor = Sequential()
	regressor.add(LSTM(units = 60, activation = 'sigmoid', return_sequences = True, input_shape = (X_train.shape[1], 10)))
	regressor.add(Dropout(0.2))
	regressor.add(LSTM(units = 60, activation = 'sigmoid', return_sequences = True))
	regressor.add(Dropout(0.2))
	regressor.add(LSTM(units = 80, activation = 'sigmoid', return_sequences = True))
	regressor.add(Dropout(0.2))
	regressor.add(LSTM(units = 120, activation = 'sigmoid'))
	regressor.add(Dropout(0.2))
	regressor.add(Dense(units = 3))
	regressor.summary()

	regressor.compile(optimizer='adam', loss = 'mean_squared_error', metrics = ['accuracy']) #find a different loss function for classification
	history = regressor.fit(X_train, y_train, epochs=100, batch_size=320)

	return (regressor,history)

In [ ]:
def My_NN_predict(data_pred, regressor):
	y_pred = regressor.predict(X_test) #predicted values for bhs
	scaler.scale_ #generated coefficient for scaling our data back to initial
	for i in range(y_pred.shape[0]):
	    #print(i,y_pred[i])
	    k = (np.argmax(y_pred[i]))
	    y_pred[i] = [0,0,0]
	    y_pred[i][k] = 1
	    #for dealing with the negative and non-format abiding output predictions

	return y_pred

In [ ]:
def Test_NN(y_pred,y_test,history):
	count = 0
	for i in range(y_pred.shape[0]):
	    if np.array_equal(y_pred[i],y_test[i]):
	        pass
	    else:
	        count+=1
	print("No of Times where our model predicted wrong output",count)
	# print(i,y_pred[i],y_test[i])
	plt.plot(history.history['loss'])
	history.history['loss']

In [ ]:
def get_training_data(data):
	data_training = data[:].copy()
	
	scaler = MinMaxScaler() #read on this from the sklearn - Rich to Rich
	data_training = scaler.fit_transform(data_training) #scales data to reduce error

	X_train = []
	y_train = []
	for i in range(30, data_training.shape[0]):
	    X_train.append(data_training[i-30:i]) #sliding window of data to help predict next day
	    y_train.append(data_training[i,[7,8,9]]) #30 chosen arbitrarily
	X_train, y_train = np.array(X_train), np.array(y_train) #converted to np array

	return (X_train, y_train)

In [ ]:
def get_testing_data(data):
	data_testing = data[8000:].copy()

	data_train = data[:8000].copy()
	past_60_days = data_train.tail(60)

	df = past_60_days.append(data_testing, ignore_index = True) #for the fist 60 values of pridict data set

	inputs = scaler.transform(df) #scaling data
	# inputs
	#inputs = df.copy()
	X_test = []
	y_test = []
	x_pred = []
	#reformatting and now feeding data into the model for prediction
	for i in range(60, inputs.shape[0]):
	    X_test.append(inputs[i-60:i])
	    y_test.append(inputs[i, [7,8,9]])
	    x_pred.append(inputs[i, [:7]])

	X_test, y_test = np.array(X_test), np.array(y_test)

	return (X_test, y_test, x_pred)

In [ ]:
def My_NN():
	data = pd.read_csv('train2.csv', date_parser = True)
	data = data.dropna()
	data = data.reset_index(drop=True)

	X_train, y_train = get_training_data(data)
	X_test, y_test, x_pred = get_testing_data(data)

	Pred_NN, Pred_history = My_NN_train(X_train, y_train)
	# Train the NN
	y_pred = My_NN_predict(X_test,Pred_NN)
	# Predict the testing data
	Test_NN(y_pred,y_test,Pred_history)
	# Test the performance of NN on testing data

	data_pred = []
	y_pred = list(y_pred)
	for i in range(X_test.shape[0]):
		data_pred.append(x_pred[i]+y_pred[i])
	with open("train3.csv","w") as csvfile:
		writer = csv.writer(csvfile)
		writer.writerow(["Open","High","Low	Close","Volume","20_day","50_day","b","h","s"])
		writer.writerow(data_pred)
	# write the predicted one hot vector (["b","h","s"]) with the information of everyday's stock price in the new csv file

In [ ]:
if __name__ == "__main__":

    result_list = []
    
    #cash = int(input("Enter Your Starting Cash: "))
    while True:
          try:
            cash = int(input("Enter Your Starting Cash: "))
            if cash > 5000:
                play = True
                break
            else:
                print(f"Please enter cash more than 5000")  
          except:
                continue
    player_obj = p.Player('AAPL', cash, {'AAPL': 0})
    node = t.Node(player=player_obj)

    trade_days = int(input("How many days would you like the AI to trade for?"))

    #AI = input("Which AI you want to choose: BaselineAI (Enter B), BeamSearchAI (Enter BS): ")

    while True:
          try:
            AI = input("Which AI you want to choose: BaselineAI (Enter B/b), BeamSearchAI (Enter BS/bs): ")
            if AI == "B" or AI == "b":
                break
            elif AI == "BS" or AI == "bs":
                break
            else:
                print(f"Enter valid choice\n")  
          except:
                continue

    if AI == 'B':
        print("you chose Baseline AI")
        t.basic_AI(node, 4, trade_days)
    else:
        print("You chose BeamSearch AI")
        t.beam_search(node, 4, trade_days)

In [ ]:
game_sizes = [5,25,45,65,85]
X_train = [] 
y_train = []
for game_size in game_sizes:
    for games in range(25):
        starting_index = random.randomint(0,len(data)-game_size-1)
        play_range_data = data[starting_index:starting_index+game_size]
        for i in range(2, play_range_data.shape[0]):
            X_train.append(data_training[i-2:i])
            y_train.append(data_training[i,[7,8,9]])
         
        regressor, hist = My_NN_train(X_train, y_train)
        
        
        

My_NN()